In [3]:
##################################################################
################### last update 20230228 #########################
##################################################################
def add_arrow_peachpuff(
    axes, line, arrow_locs=[0.2, 0.4, 0.6, 0.8],
    arrowstyle='-|>', arrowsize=1, transform=None):
    if not isinstance(line, mlines.Line2D):
        raise ValueError("expected a matplotlib.lines.Line2D object")
    x, y = line.get_xdata(), line.get_ydata()

    arrow_kw = {
        "arrowstyle": arrowstyle,
        "mutation_scale": 10 * arrowsize,
    }

    color = 'peachpuff'
    #line.get_color()
    use_multicolor_lines = isinstance(color, np.ndarray)
    if use_multicolor_lines:
        raise NotImplementedError("multicolor lines not supported")
    else:
        arrow_kw['color'] = color

    linewidth = line.get_linewidth()
    if isinstance(linewidth, np.ndarray):
        raise NotImplementedError("multiwidth lines not supported")
    else:
        arrow_kw['linewidth'] = linewidth

    if transform is None:
        transform = axes.transData

    arrows = []
    for loc in arrow_locs:
        s = np.cumsum(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2))
        n = np.searchsorted(s, s[-1] * loc)
        arrow_tail = (x[n], y[n])
        arrow_head = (np.mean(x[n:n + 2]), np.mean(y[n:n + 2]))
        p = mpatches.FancyArrowPatch(
            arrow_tail, arrow_head, transform=transform,
            **arrow_kw)
        axes.add_patch(p)
        arrows.append(p)
    return arrows

def time_1D(x, number_of_raters):
    size=[500/number_of_raters]
    for i in range(len(x)-1):
        if x[i]==x[i+1]:
            size.append(size[i]+500/number_of_raters)
        else:
            size.append(500/number_of_raters)
    return size



#---------------------- 1D ASSG -----------------------
def plot_1D_ASSG(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(2*(5.4-0.6),2*(number_of_raters+2)*1.1))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,v[j], ratermeanvaluelist(v))
    #yfit_yconst=np.linspace(0, number_of_raters+0.5)
    #xfit_yconst=popty[0]+0*yfit_yconst
    #ax.plot(xfit_yconst, yfit_yconst, color='crimson')
        
#mean value of all ratings------------------------------------------------------------
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+number_of_raters+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='RdPu', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='hotpink', 
                                 label='mean values of raters \n$\overline{\Delta rating}=$%.2f$\pm$%.2f'
                                 %(meanlist(error_ratings), stdlist(error_ratings)),
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='RdPu', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_hotpink(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    ax.scatter(popty[0], number_of_raters+1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=20)
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdx=np.linspace(popty[0]-meanlist(meanvalue_sd), popty[0]+meanlist(meanvalue_sd), 50)
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=15)
    #color errorbars
    errorbar_color = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    cmap = cm.RdPu
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings,
                 fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings, fmt='none',
                 ecolor=cmap(norm(errorbar_color)), elinewidth=7, zorder=8)
     
    
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+2)*1.1])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    ax.set_ylabel('Rater', fontsize='24')
###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='17').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

    